# Preview the hoof scans for Elke
Let's see what we did there...

In [ ]:
import platform
import os
import glob
import pandas
import imageio
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import seaborn
import dask
import dask_image.imread
import numpy
from dask.distributed import Client, LocalCluster
from tqdm.auto import tqdm, trange

In [ ]:
# Import our own parsing functions which we've added as submodule
from BrukerSkyScanLogfileRuminator.parsing_functions import *

In [ ]:
# # Code linting
# %load_ext pycodestyle_magic

In [ ]:
# %pycodestyle_on

In [ ]:
# Set dask temporary folder
# Do this before creating a client: https://stackoverflow.com/a/62804525/323100
import tempfile
if 'Linux' in platform.system():
    tmp = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
elif 'Darwin' in platform.system():
    tmp = tempfile.gettempdir()
else:
    if 'anaklin' in platform.node():
        tmp = os.path.join('F:\\')
    else:
        tmp = os.path.join('D:\\')
dask.config.set({'temporary_directory': os.path.join(tmp, 'tmp')})
print('Dask temporary files go to %s' % dask.config.get('temporary_directory'))

In [ ]:
# Start cluster and client now, after setting tempdir
client = Client()

In [ ]:
client

In [ ]:
# # Ignore warnings in the notebook
# import warnings
# warnings.filterwarnings("ignore")

In [ ]:
# Set up figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation
plt.rcParams['figure.figsize'] = (16, 9)  # Size up figures a bit
plt.rcParams['figure.dpi'] = 300

In [ ]:
# Setup scale bar defaults
plt.rcParams['scalebar.location'] = 'lower right'
plt.rcParams['scalebar.frameon'] = False
plt.rcParams['scalebar.color'] = 'white'

In [ ]:
# Display all plots identically
lines = 3
# And then do something like
# plt.subplot(lines, int(numpy.ceil(len(Data) / float(lines))), c + 1)

In [ ]:
# Different locations if running either on Linux or Windows
FastSSD = True
# to speed things up significantly
if 'Linux' in platform.system():
    if FastSSD:
        BasePath = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
    else:
        BasePath = os.path.join(os.sep, 'home', 'habi', '2214')
elif 'Darwin' in platform.system():
    # First mount smb://resstore.unibe.ch/ana_rs_djonov/data in the Finder
    FastSSD = False
    BasePath = os.path.join('/Volumes/data/')
elif 'Windows' in platform.system():
    if FastSSD:
        BasePath = os.path.join('F:\\')
    else:
        if 'anaklin' in platform.node():
            BasePath = os.path.join('N:\\')
        else:
            BasePath = os.path.join('V:\\')
Root = os.path.join(BasePath, 'VETSUISSE', 'Laminitis')
print('We are loading all the data from %s' % Root)

In [ ]:
def get_git_hash():
    '''
    Get the current git hash from the repository.
    Based on http://stackoverflow.com/a/949391/323100 and
    http://stackoverflow.com/a/18283905/323100
    '''
    from subprocess import Popen, PIPE
    import os
    gitprocess = Popen(['git',
                        '--git-dir',
                        os.path.join(os.getcwd(), '.git'),
                        'rev-parse',
                        '--short',
                        '--verify',
                        'HEAD'],
                       stdout=PIPE)
    (output, _) = gitprocess.communicate()
    return output.strip().decode("utf-8")

In [ ]:
# # Make directory for output
# OutPutDir = os.path.join(os.getcwd(), 'Output', get_git_hash())
# print('We are saving all the output to %s' % OutPutDir)
# os.makedirs(OutPutDir, exist_ok=True)

In [ ]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [ ]:
# Get *all* log files, unsorted but faster than with glob
print('Searching for all log files in %s' % Root)
Data['LogFile'] = [os.path.join(root, name)
                   for root, dirs, files in os.walk(Root)
                   for name in files
                   if name.endswith((".log"))]

In [ ]:
# Get all folders
Data['Folder'] = [os.path.dirname(f) for f in Data['LogFile']]

In [ ]:
Data

In [ ]:
#Check for samples which are not yet reconstructed
for c, row in Data.iterrows():
    # Iterate over every 'proj' folder
    if 'proj' in row.Folder:
        if not 'TScopy' in row.Folder and not 'PR' in row.Folder:
            # If there's nothing with 'rec*' on the same level, then tell us        
            if not glob.glob(row.Folder.replace('proj', '*rec*')):
                print('- %s is missing matching reconstructions' % row.LogFile[len(Root)+1:])

In [ ]:
Data['XYAlignment'] = [glob.glob(os.path.join(f, '*.csv')) for f in Data['Folder']]

In [ ]:
# Check for samples which are missing the .csv-files for the XY-alignment
for c, row in Data.iterrows():
    # Iterate over every 'proj' folder
    if 'proj' in row.Folder:
        if not len(row.XYAlignment):
            if not any(x in row.LogFile for x in ['rectmp.log']):
                # 'rectmp.log' because we only exclude it afterwards :)
                print('- %s has *not* been X/Y aligned' % row.LogFile[len(Root)+1:])

- "Limb03\proj\" is the scan where the filament broke.
- "Limb05\proj_orig\" are the original projections of the scan where the control PC crashed at segment 3 of 3.
It's "OK" that these scans have not been aligned.

In [ ]:
# Get rid of all logfiles that we don't want
for c, row in Data.iterrows():
    if 'rec' not in row.Folder:  # drop all non-rec folders
        Data.drop([c], inplace=True)
    elif 'SubScan' in row.Folder:  # drop all partial reconstructions which might be there from synchronization
        Data.drop([c], inplace=True)        
    elif 'rectmp.log' in row.LogFile:  # drop all temporary logfiles
        Data.drop([c], inplace=True)
# Reset dataframe to something that we would get if we only would have loaded the 'rec' files
Data = Data.reset_index(drop=True)

In [ ]:
# Generate us some meaningful colums
Data['Sample'] = [l[len(Root)+1:].split(os.sep)[-3] for l in Data['LogFile']]
Data['Scan'] = ['_'.join(l[len(Root)+1:].split(os.sep)[1:-1]) for l in Data['LogFile']]

In [ ]:
Data[['Sample', 'Scan']]

In [ ]:
# Get the file names of the reconstructions
Data['Reconstructions'] = [sorted(glob.glob(os.path.join(f, '*rec0*.png'))) for f in Data['Folder']]
Data['Number of reconstructions'] = [len(r) for r in Data.Reconstructions]

In [ ]:
# Drop samples which have either not been reconstructed yet or of which we deleted the reconstructions with
# `find . -name "*rec*.png" -type f -mtime +333 -delete`
# Based on https://stackoverflow.com/a/13851602
# for c,row in Data.iterrows():
#     if not row['Number of reconstructions']:
#         print('%s contains no PNG files, we might be currently reconstructing it' % row.Folder)
Data = Data[Data['Number of reconstructions'] > 0]
Data.reset_index(drop=True, inplace=True)
print('We have %s folders containing reconstructions' % (len(Data)))

In [ ]:
# Get scanning parameters to doublecheck from logfiles
Data['Scanner'] = [scanner(log) for log in Data['LogFile']]
Data['Voltage'] = [voltage(log) for log in Data['LogFile']]
Data['Current'] = [current(log) for log in Data['LogFile']]
Data['Voxelsize'] = [pixelsize(log, rounded=True) for log in Data['LogFile']]
Data['ProjectionSize'] = [projection_size(log) for log in Data['LogFile']]
Data['Exposuretime'] = [exposure(log) for log in Data['LogFile']]
Data['Averaging'] = [averaging(log) for log in Data['LogFile']]
Data['Stacks'] = [stacks(log) for log in Data['LogFile']]
Data['RotationStep'] = [rotationstep(log) for log in Data['LogFile']]
Data['Scan date'] = [scandate(log) for log in Data['LogFile']]
Data['Scan time'] = [duration(log) for log in Data['LogFile']]

In [ ]:
Data.Voltage.unique()

In [ ]:
Data.Current.unique()

In [ ]:
Data.Averaging.unique()

In [ ]:
Data.Voxelsize.unique()

In [ ]:
# Get reconstruction parameters to doublecheck from logfiles
Data['Grayvalue'] = [reconstruction_grayvalue(log) for log in Data['LogFile']]
Data['RingartefactCorrection'] = [ringremoval(log) for log in Data['LogFile']]
Data['BeamHardeningCorrection'] = [beamhardening(log) for log in Data['LogFile']]
Data['DefectPixelMasking'] = [defectpixelmasking(log) for log in Data['LogFile']]

In [ ]:
Data['Grayvalue'].unique()

In [ ]:
Data['RingartefactCorrection'].unique()

In [ ]:
# Sort our dataframe by scan date
Data.sort_values(by='Scan date', inplace=True, ignore_index=True)

In [ ]:
for c,row in Data.iterrows():
    if row.Grayvalue != 0.1:
        print(row.Sample, row.Grayvalue)

In [ ]:
for c,row in Data.iterrows():
    if not row.RingartefactCorrection:
        print(row.Sample, row.RingartefactCorrection)

In [ ]:
Data[['Sample', 'Scan', 'Grayvalue', 'RingartefactCorrection', 'BeamHardeningCorrection', 'DefectPixelMasking']]

Some consistency checks

In [ ]:
# Check ringremoval parameters
for machine in Data['Scanner'].unique():
    print('For the %s we have '
          'ringartefact-correction values of %s' % (machine,
                                                    Data[Data.Scanner==machine]['RingartefactCorrection'].unique()))

In [ ]:
# Check beamhardening parameters
for scanner in Data.Scanner.unique():
    print('For the %s we have '
          'beamhardening correction values of %s' % (scanner,
                                                     Data[Data.Scanner==scanner]['BeamHardeningCorrection'].unique()))

In [ ]:
# Check defect pixel masking parameters
for scanner in Data.Scanner.unique():
    print('For the %s we have '
          'defect pixel masking values of %s' % (scanner,
                                                 Data[Data.Scanner==scanner]['DefectPixelMasking'].unique()))

In [ ]:
# Check defect pixel masking parameters
for scanner in Data.Scanner.unique():
    print('For the %s we have '
          'reconstruction gray values of %s' % (scanner,
                                                Data[Data.Scanner==scanner]['Grayvalue'].unique()))

Check and display scan times

In [ ]:
Data['Scan time total'] = [ st * stk  for st, stk in zip(Data['Scan time'], Data['Stacks'])]

In [ ]:
# # https://www.geeksforgeeks.org/iterating-over-rows-and-columns-in-pandas-dataframe/
# columns = list(Data)
# columns.remove('Folder') 
# columns.remove('Sample')
# columns.remove('LogFile')
# columns.remove('Reconstructions')
# columns.remove('Number of reconstructions')
# columns.remove('Grayvalue')
# columns.remove('Scan time')
# columns.remove('Scan time total')
# columns.remove('Scan date')
# print(columns)
# for col in columns:
#     print(col)
#     print(Data[col].unique())
#     print(80*'-')    

In [ ]:
# Check voxel sizes (*rounded* to two after-comma values)
# If different, spit out which values
roundto = 2
if len(Data['Voxelsize'].round(roundto).unique()) > 1:
    print('We scanned all datasets with %s different voxel sizes' % len(Data['Voxelsize'].round(roundto).unique()))
    for vs in sorted(Data['Voxelsize'].round(roundto).unique()):
        print('-', vs, 'um for ', end='')
        for c, row in Data.iterrows():
            if float(vs) == round(row['Voxelsize'], roundto):
                print(os.path.join(row['Sample'], row['Scan']), end=', ')
        print('')
else:
    print('We scanned all datasets with equal voxel size, namely %s um.' % float(Data['Voxelsize'].round(roundto).unique()))

In [ ]:
Data[['Sample', 'Scan',
      'BeamHardeningCorrection', 'DefectPixelMasking',
      'RingartefactCorrection', 'Grayvalue', 'Voxelsize']]

In [ ]:
# if len(Data['Grayvalue'].unique()) > 1:
#     print('We reconstructed the datasets with different maximum gray values, namely')
#     for gv in Data['Grayvalue'].unique():
#         print(gv, 'for Samples ', end='')
#         for c, row in Data.iterrows():
#             if float(gv) == row['Grayvalue']:
#                 print(os.path.join(row['Sample'], row['Scan']), end=', ')
#         print('')
# else:
#     print('We reconstructed all datasets with equal maximum gray value, namely %s.' % Data['Grayvalue'].unique()[0])

In [ ]:
# Data[['Sample', 'Scan',
#       'Voxelsize', 'Scanner',
#       'Scan date', 'CameraWindow', 'RotationStep', 'Averaging',
#       'Scan time', 'Stacks', 'Scan time total']]

In [ ]:
Data['Scan time'][0]

In [ ]:
# Get an overview over the total scan time
# Nice output based on https://stackoverflow.com/a/8907407/323100
total_seconds = int(Data['Scan time total'].sum())
hours, remainder = divmod(total_seconds,60*60)
minutes, seconds = divmod(remainder,60)
print('In total, we scanned for %s hours and %s minutes)' % (hours, minutes))
for machine in Data['Scanner'].unique():
    total_seconds = int(Data[Data['Scanner'] == machine]['Scan time total'].sum())
    hours, remainder = divmod(total_seconds,60*60)
    minutes, seconds = divmod(remainder,60)
    print('\t - Of these, we scanned %s hours and %s minutes on the %s,'
          ' for %s scans' % (hours,
                             minutes,
                             machine,
                             len(Data[Data['Scanner'] == machine])))

In [ ]:
Data[['Sample', 'Scan',
      'Voxelsize', 'Scanner',
      'Scan date', 'ProjectionSize', 'RotationStep', 'Averaging', 'Scan time', 'Stacks' ]].to_excel('ScanningDetails.xlsx')

In [ ]:
Data[['Sample', 'Scan',
      'Voxelsize', 'Scanner',
      'Scan date', 'ProjectionSize',
      'RotationStep', 'Averaging', 'Scan time', 'Stacks' ]].to_excel(os.path.join(Root,'ScanningDetails.xlsx'))

In [ ]:
Data.tail()

In [ ]:
Data['PreviewImagePath'] = [sorted(glob.glob(os.path.join(f, '*_spr.bmp')))[0] for f in Data['Folder']]
Data['PreviewImage'] = [dask_image.imread.imread(pip)
                        if pip
                        else numpy.random.random((100, 100)) for pip in Data['PreviewImagePath']]

In [ ]:
# Make an approximately square overview image
lines = 2

In [ ]:
Data.Sample

In [ ]:
for c, row in Data.iterrows():
    plt.subplot(lines, int(numpy.ceil(len(Data) / float(lines))), c + 1)
    plt.imshow(row.PreviewImage.squeeze())
    plt.title('%s' % os.path.join(row['Sample'], row['Scan']))
    plt.gca().add_artist(ScaleBar(row['Voxelsize'],
                                  'um',
                                  color='black',
                                  frameon=True))
    plt.axis('off')
plt.tight_layout()
plt.savefig(os.path.join(Root, 'ScanOverviews.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
# Load all reconstructions into ephemereal DASK arrays
Reconstructions = [None] * len(Data)
for c, row in tqdm(Data.iterrows(),
                   desc='Load reconstructions',
                   total=len(Data)):
    Reconstructions[c] = dask_image.imread.imread(os.path.join(row['Folder'],
                                                               '*rec*.png'))

In [ ]:
# Check if something went wrong
# for file in Data['OutputNameRec']:
#     print(file)
#     dask.array.from_zarr(file)

In [ ]:
# How big are the datasets?
Data['Size'] = [rec.shape for rec in Reconstructions]

In [ ]:
# The three cardinal directions
directions = ['Axial',
              'Coronal',
              'Sagittal']

In [ ]:
# Read or calculate the middle slices, put them into the dataframe and save them to disk
for d, direction in enumerate(directions):
    Data['Mid_' + direction] = [None] * len(Reconstructions)
for c, row in tqdm(Data.iterrows(), desc='Central images', total=len(Data), leave=False):
    for d, direction in tqdm(enumerate(directions),
                             desc='%s/%s' % (row['Sample'], row['Scan']),
                             leave=False,
                             total=len(directions)):
        outfilepath = os.path.join(os.path.dirname(row['Folder']),
                                   '%s.%s.Central.%s.png' % (row['Sample'],
                                                            row['Scan'],
                                                            direction))
        if not os.path.exists(outfilepath):
            # Generate requested axial view
            if 'Axial' in direction:
                mid = Reconstructions[c][Data['Size'][c][0] // 2]
            if 'Sagittal' in direction:
                mid = Reconstructions[c][:, Data['Size'][c][1] // 2, :]
            if 'Coronal' in direction:
                mid = Reconstructions[c][:, :, Data['Size'][c][2] // 2]
            # Write the calculated 'direction' view to disk
            imageio.imwrite(outfilepath, mid[:,:,0])
        Data.at[c, 'Mid_' + direction] = dask_image.imread.imread(outfilepath).squeeze()            

In [ ]:
for s in Data.Scan:
    print(s.split('_'))

In [ ]:
# Show middle slices
for c, row in tqdm(Data.iterrows(),
                   desc='Saving central images overview',
                   total=len(Data),
                   leave=False):
    outfilepath = os.path.join(os.path.dirname(row['Folder']),
                               '%s.%s.CentralSlices.png' % (row['Sample'], row['Scan']))
    if not os.path.exists(outfilepath):    
        for d, direction in tqdm(enumerate(directions),
                                 desc='%s/%s' % (row['Sample'], row['Scan']),
                                 leave=False,
                                 total=len(directions)):
            plt.subplot(1, 3, d + 1)
            plt.imshow(row['Mid_' + direction].squeeze())
            if d == 0:
                plt.axhline(row.Size[1] // 2, c=seaborn.color_palette()[0])
                plt.axvline(row.Size[2] // 2, c=seaborn.color_palette()[1])
                plt.gca().add_artist(ScaleBar(row['Voxelsize'],
                                              'um',
                                              color=seaborn.color_palette()[2]))
            elif d == 1:
                plt.axhline(row.Size[0] // 2, c=seaborn.color_palette()[2])
                plt.axvline(row.Size[d] // 2, c=seaborn.color_palette()[1])
                plt.gca().add_artist(ScaleBar(row['Voxelsize'],
                                              'um',
                                              color=seaborn.color_palette()[0]))
            else:
                plt.axhline(row.Size[0] // 2, c=seaborn.color_palette()[2])
                plt.axvline(row.Size[d] // 2, c=seaborn.color_palette()[0])
                plt.gca().add_artist(ScaleBar(row['Voxelsize'],
                                              'um',
                                              color=seaborn.color_palette()[1]))
            plt.title('%s %s' % (os.path.join(row['Sample'], row['Scan']),
                                 direction + ' Central slice'))
            plt.axis('off')
            plt.savefig(outfilepath,
                        transparent=True,
                        bbox_inches='tight')
        plt.show()

In [ ]:
# Read or calculate the directional MIPs, put them into the dataframe and save them to disk
for d, direction in enumerate(directions):
    Data['MIP_' + direction] = [None] * len(Reconstructions)
for c, row in tqdm(Data.iterrows(), desc='MIPs', total=len(Data), leave=False):
    for d, direction in tqdm(enumerate(directions),
                             desc='%s/%s' % (row['Sample'], row['Scan']),
                             leave=False,
                             total=len(directions)):
        outfilepath = os.path.join(os.path.dirname(row['Folder']),
                                   '%s.%s.MIP.%s.png' % (row['Sample'],
                                                      row['Scan'],
                                                      direction))
        if not os.path.exists(outfilepath):
            # Generate MIP and write it to disk
            mip = Reconstructions[c].max(axis=d)
            imageio.imwrite(outfilepath, mip[:,:,0])
        Data.at[c, 'MIP_' + direction] = dask_image.imread.imread(outfilepath).squeeze()

In [ ]:
# Show MIP slices
for c, row in tqdm(Data.iterrows(),
                   desc='Saving MIP images overview',
                   total=len(Data),
                   leave=False):
    outfilepath = os.path.join(os.path.dirname(row['Folder']),
                               '%s.%s.MIPs.png' % (row['Sample'], row['Scan']))
    if not os.path.exists(outfilepath):    
        for d, direction in tqdm(enumerate(directions),
                                          desc='%s/%s' % (row['Sample'], row['Scan']),
                                          leave=False,
                                          total=len(directions)):
            plt.subplot(1, 3, d + 1)
            plt.imshow(row['MIP_' + direction])
            plt.gca().add_artist(ScaleBar(row['Voxelsize'],
                                          'um'))
            plt.title('%s %s' % (os.path.join(row['Sample'], row['Scan']),
                                 direction + ' MIP'))
            plt.axis('off')
        plt.savefig(outfilepath,
                    transparent=True,
                    bbox_inches='tight')
        plt.show()

In [ ]:
# Calculate the histograms of one of the MIPs (or reconstructions, depending on which line we comment)
# Caveat: dask.da.histogram returns histogram AND bins, making each histogram a 'nested' list of [h, b]
Data['Histogram'] = [dask.array.histogram(dask.array.array(mip.squeeze()),
                                          bins=2**8,
                                          range=[0, 2**8]) for mip in Data['MIP_Coronal']]  
# Data['Histogram'] = [dask.array.histogram(rec,
#                                           bins=2**8,
#                                           range=[0, 2**8]) for rec in Reconstructions]
# Actually compute the data and put only h into the dataframe, since we use it quite often below.
# Discard the bins
Data['Histogram'] = [h.compute() for h,b in Data['Histogram']]

In [ ]:
for c, row in sorted(Data.iterrows()):
    plt.semilogy(row.Histogram, label='%s/%s' % (row.Sample, row.Scan))
plt.xlim([0, 255])
plt.legend()
plt.show()

In [ ]:
numpy.unique(Data['MIP_Axial'][0].compute())

In [ ]:
def overeexposecheck(item, threshold=222, howmanypercent=0.025, whichone='Coronal', verbose=False):
    '''Function to check if a certain amount of voxels are brighter than a certain value'''
    if (Data['MIP_%s' % whichone][item]>threshold).sum() > (Data['MIP_%s' % whichone][item].size * howmanypercent / 100):
        if verbose:
            plt.imshow(Data['MIP_%s' % whichone][item])
            plt.imshow(dask.array.ma.masked_less(Data['MIP_%s' % whichone][item], threshold),
                       cmap='viridis_r',
                       alpha=.618)
            plt.title('%s/%s\n%s px of %s Mpixels (>%s%%) are brighter '
                      'than %s' % (Data['Sample'][item],
                                   Data['Scan'][item],
                                   (Data['MIP_%s' % whichone][item]>threshold).sum().compute(),
                                   round(1e-6 * Data['MIP_%s' % whichone][item].size,2),
                                   howmanypercent,
                                   threshold))
            plt.axis('off')
            plt.gca().add_artist(ScaleBar(Data['Voxelsize'][item],
                                          'um'))
            plt.show()
        return(True)
    else:
        return(False)    

In [ ]:
# Check if 'too much' of the MIP is overexposed
Data['OverExposed'] = [overeexposecheck(c,
                                        whichone='Axial',
                                        verbose=True) for c, row in Data.iterrows()]

In [ ]:
print('At the moment, we have previewed %s scans of %s samples in %s' % (len(Data),
                                                                         len(Data.Sample.unique()),
                                                                         Root))